# The linear elasticity problem

Let $\Omega \subset \mathbb{R}^3$ and consider the linear elasticity problem with the following strong formulation:
The governing equations for small elastic deformations of a body $ \Omega $ can be expressed as:

$$
\begin{align}
    -\nabla \cdot \sigma(u) &= f & \text{in } \Omega \\
    \sigma(u) &= \kappa \text{tr}(\epsilon(u))I + 2 \mu \epsilon(u)
\end{align}
$$

where $ \sigma $ is the stress tensor, $ f $ represents the body force per unit volume, $ \kappa $ and $ \mu $ are Lamé's elasticity parameters for the material, $ I $ denotes the identity tensor, $ \epsilon $ is the symmetric strain tensor and the displacement vector field is denoted by $ u $. $ \epsilon := \frac{1}{2}(\nabla u + (\nabla u)^T) $
